# Sprawozdanie z laboratorium 2

***Autor: Adam Dąbkowski***

Celem drugiego laboratorium jest zaimplementowanie algorytmu genetycznego z selekcją ruletkową, krzyżowaniem jednopunktowym oraz sukcesją generacyjną. Dodatkowo należy zbadać wpływ doboru parametrów na przykładzie danego problemu.

## 0. Importowanie niezbędnych bibliotek

In [1]:
import random
import copy
from utils import *

## 1. Tworzenie populacji

In [2]:
class PopulationCreator:
    def __init__(self, population_size, individual_size):
        self._population_size = population_size
        self._individual_size = individual_size

    def create_population(self):
        population = []

        for i in range(self._population_size):
            individual = []
            for j in range(self._individual_size):
                individual.append(random.randint(0, 1))
            population.append(np.array(individual))

        return np.array(population)

## 2. Zdefiniowanie problemu (funkcji celu)

In [3]:
def problem(x):
    h0 = 200
    v0 = 0
    fuel = x.sum()
    m0 = 200 + fuel

    for i in x:
        m0 -= i * 1
        a = (45/m0) * i - 0.09
        v0 = v0 + a
        h0 = h0 + v0

        if (0 <= h0 < 2) and abs(v0) < 2:
            return 2000 - fuel

        elif h0 < 0:
            return -1000 - fuel

    return - fuel

## 3. Implementacja algorytmu genetycznego

Głównym zadaniem drugiego laboratorium jest implementacja algorytmu genetycznego. W tym celu stworzona została klasa ***GeneticAlgorithmSolver***. Podczas tworzenia obiektu tej klasy istnieje możliwość podania czterech parametrów:
- ***t_max*** - liczba iteracji (domyślna wartość wynosi $1000$)
- ***pc*** - prawdopodobieństwo krzyżowania (domyślna wartość wynosi $0,5$)
- ***pm*** - prawdopodobieństwo mutacji (domyślna wartość wynosi $0,05$)
- ***u*** - liczba osobników w populacji (domyślna wartość wynosi $250$)

Klasa ***GeneticAlgorithmSolver*** zawiera także sześć metod:
- ***get_parameters()*** - metoda ta zwraca wartości parametrów
- ***solve()*** - metoda zawierająca w sobie algorytm genetyczny, wykorzystuje metody ***score()*** , ***find_best()*** , ***selection()*** , ***crossing_and_mutation()*** . W wyniku jej wywołania otrzymujemy najlepszego osobnika i wartość funkcji celu
- ***score()*** - metoda zwracająca wyniki dla osobników w populacji
- ***find_best()*** - metoda zwracająca najlepszego osobnika
- ***selection()*** - metoda, której zadaniem jest wybranie lepszych osobników z większym prawdopodobieństwem
- ***crossing_and_mutation()*** - metoda odpowiedzialna za krzyżowanie i mutację

In [4]:
class GeneticAlgorithmSolver:
    def __init__(self, t_max=1000, pc=0.5, pm=0.05, u=250):
        self._t_max = t_max
        self._pc = pc
        self._pm = pm
        self._u = u

    def get_parameters(self):
        return {
            "t_max": self._t_max,
            "pc": self._pc,
            "pm": self._pm,
            "u": self._u
        }

    def solve(self, problem, pop0, printing=False):
        t = 0
        P = pop0
        O = self.score(problem, P)
        x_best, o_best = self.find_best(P, O)
        t_best = 0
        while t < self._t_max:
            S = self.selection(P, O, self._u)
            M = self.crossing_and_mutation(S, self._pc, self._pm, self._u)
            O = self.score(problem, M)
            x, o = self.find_best(M, O)

            if o > o_best:
                x_best = x
                o_best = o
                t_best = t

            P = M
            t += 1

            if printing:
                print(f"Iteration {t}: Best score = {o}, Best score to date = {o_best}")

        if printing:
            print(f" ")
            print(f"Best score = {o_best}, Iteration {t_best}")

        return x_best, o_best

    @staticmethod
    def score(problem, P):
        return [problem(x) for x in P]

    @staticmethod
    def find_best(P, O):
        index = np.argmax(O)
        return P[index], O[index]

    @staticmethod
    def selection(P, O, u):
        O = [o + (1000 + P.shape[1]) for o in O]
        O = [o / sum(O) for o in O]
        indices = np.arange(0, P.shape[0])
        selected_indices = np.random.choice(indices, u, replace=True, p=O)

        return P[selected_indices]

    @staticmethod
    def crossing_and_mutation(S, pc, pm, u):
        M = []

        # crossing
        for i in range(0, u - 1, 2):
            individual_1 = S[i]
            individual_2 = S[i + 1]

            if random.random() < pc:
                crossing_point = random.randint(1, len(individual_1))
                individual_1_crossed = copy.deepcopy(individual_1)
                individual_2_crossed = copy.deepcopy(individual_2)

                individual_1_crossed[crossing_point:] = individual_2[crossing_point:]
                individual_2_crossed[crossing_point:] = individual_1[crossing_point:]

                M.append(individual_1_crossed)
                M.append(individual_2_crossed)

            else:
                M.append(individual_1)
                M.append(individual_2)



        if u % 2 == 1:
            M.append(S[-1])

        # mutation
        for individual in M:
            for i in range(individual.shape[0]):
                if random.random() < pm:
                    individual[i] = 1 if individual[i] == 0 else 0
        return np.array(M)

## 4. Zastosowanie algorytmu genetycznego w celu wyznaczenia najlepszego osobnika

Po zaimplementowaniu algorytmu genetycznego, a także po zdefiniowaniu funkcji celu możemy przejść do testowania solvera w celu znalezienia największej wartości wspomnianej funkcji. Na początku wykorzystany zostanie solver z domyślnymi wartościami, jednakże należy najpierw utworzyć populację początkową. W naszych doświadczeniach liczba osobników w populacji początkowej będzie wynosiła $250$, natomiast wielkość osobnika będzie równa $200$.

In [32]:
population_creator = PopulationCreator(population_size=250, individual_size=200)

In [33]:
pop0 = population_creator.create_population()

Następnie przechodzimy do stworzenia solvera.

In [34]:
solver = GeneticAlgorithmSolver()

Chcąc upewnić się jakie parametry ma nasz solver możemy wykorzystać metodę ***get_parameters()*** .

In [35]:
solver.get_parameters()

{'t_max': 1000, 'pc': 0.5, 'pm': 0.05, 'u': 250}

Teraz możemy już przejść do rozwiązania naszego problemu.

In [36]:
x_best, o_best = solver.solve(problem, pop0, printing=True)

Iteration 1: Best score = 1914, Best score to date = 1917
Iteration 2: Best score = 1916, Best score to date = 1917
Iteration 3: Best score = 1918, Best score to date = 1918
Iteration 4: Best score = 1917, Best score to date = 1918
Iteration 5: Best score = 1920, Best score to date = 1920
Iteration 6: Best score = 1919, Best score to date = 1920
Iteration 7: Best score = 1919, Best score to date = 1920
Iteration 8: Best score = 1920, Best score to date = 1920
Iteration 9: Best score = 1915, Best score to date = 1920
Iteration 10: Best score = 1917, Best score to date = 1920
Iteration 11: Best score = 1916, Best score to date = 1920
Iteration 12: Best score = 1916, Best score to date = 1920
Iteration 13: Best score = 1918, Best score to date = 1920
Iteration 14: Best score = 1915, Best score to date = 1920
Iteration 15: Best score = 1916, Best score to date = 1920
Iteration 16: Best score = 1918, Best score to date = 1920
Iteration 17: Best score = 1914, Best score to date = 1920
Iterat

In [37]:
x_best

array([0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0])

Uzyskany wynik to $1930$, który otrzymaliśmy w $288$ iteracji. Wynik ten możemy uznać za bardzo dobry, jednakże nie powinniśmy oceniać doboru hiperparametrów jedynie na podstawie jednego przebiegu. Dlatego w dalszej części sprawozdania nasze wnioski będziemy opierać na wskaźnikach jakości, które pochodzą z analizy zawierającej wiele przebiegów o tych samych hiperparametrach.

## 5. Zastosowanie algorytmu genetycznego w celu wyznaczenia najlepszego osobnika - Badanie wpływu prawdopodobieństwa mutacji


Aby usprawnić analizę otrzymanych wyników, zaimplementowana została klasa ***GeneticAlgorithmAnalyzer***, która podczas tworzenia obiektu przyjmuje jako argumenty **solver**, **funkcję celu**, **generator populacji** oraz **liczbę przebiegów**, na podstawie których będziemy mogli ocenić dobór parametrów solvera, na podstawie zwracanych wartości takich jak: **wartość maksymalna**, **wartość minimalna**, **średnia wyników**, **mediana wyników** oraz **odchylenie standardowe**.

Na początku przetestujemy solver z domyślnymi wartościami, tym razem dla wielu przebiegów.

In [75]:
solver = GeneticAlgorithmSolver()

In [76]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator ,problem=problem, n_samples=30)

In [77]:
analyzer.analyze()

Sample 1: Best score = 1925
Sample 2: Best score = 1932
Sample 3: Best score = 1932
Sample 4: Best score = 1927
Sample 5: Best score = 1926
Sample 6: Best score = 1926
Sample 7: Best score = 1927
Sample 8: Best score = 1927
Sample 9: Best score = 1927
Sample 10: Best score = 1924
Sample 11: Best score = 1926
Sample 12: Best score = 1925
Sample 13: Best score = 1927
Sample 14: Best score = 1925
Sample 15: Best score = 1925
Sample 16: Best score = 1925
Sample 17: Best score = 1928
Sample 18: Best score = 1925
Sample 19: Best score = 1926
Sample 20: Best score = 1927
Sample 21: Best score = 1926
Sample 22: Best score = 1925
Sample 23: Best score = 1927
Sample 24: Best score = 1925
Sample 25: Best score = 1925
Sample 26: Best score = 1929
Sample 27: Best score = 1930
Sample 28: Best score = 1926
Sample 29: Best score = 1927
Sample 30: Best score = 1926
 
Best sample = 2
Best score = 1932, Minimum score = 1924
Average score = 1926.6, Median score = 1926.0, Standard deviation = 1.92527054375

Jak widać powyżej, wartość minimalna różni się od maksymalnej, która w jednym z przebiegów wynosi aż $1932$. Fakt ten potwierdza to, jak istotny jest sposób oceny doboru hiperparametrów nie na podstawie jednego przebiegu, a na ich znacznej ilości.

Średnia otrzymanych wyników jest równa $1926,6$, co jest wartością zbliżoną do otrzymanej mediany wyników wynoszącej $1926$. Odchylenie standardowe wynosi w przybliżeniu $1,9253$, które, mając na uwadze otrzymane wyniki, jest stosunkowo małe. Jednakże w chwili obecnej nie jesteśmy w stanie stwierdzić, czy rzeczywiście tak jest, ponieważ nie wiemy, jak algorytm zadziała dla innych zestawów hiperparametrów. Pewni natomiast możemy być tego, że w każdym przebiegu rakiecie udało się wylądować, o czym świadczy wynik bliski wartości $2000$. Sprawdźmy jednak, czy uda nam się zredukować koszty startu rakiety tak, aby jeszcze bardziej zbliżyć się do wartości równej $2000$, zmieniając dobór hiperparametrów. Jednym z nich jest prawdopodobieństwo mutacji $pm$. O tym, jak jego wartość wpływa na otrzymane wyniki, przekonamy się w poniższych doświadczeniach.


#### 5.1 Badanie wpływu prawdopodobieństwa mutacji dla $t_{max} = 1000$

Aby końcowe wnioski były względnie uniwersalne, nasze rozważania zostaną przeprowadzone dla różnej liczby iteracji $t_{max}$ w danym przebiegu, przy stałej wielkości populacji równej $250$. Jako pierwsza zostanie przeprowadzona analiza dla domyślnej wartości $t_{max} = 1000$.

Aby algorytm genetyczny działał jak najlepiej, zarówno prawdopodobieństwo krzyżowania, jak i prawdopodobieństwo mutacji muszą być dobrane w odpowiedni sposób. Na początku sprawdźmy, jak zachowa się algorytm, jeśli prawdopodobieństwo mutacji zostanie zwiększone. Dzięki temu zabiegowi zwiększymy różnorodność osobników w populacji. Być może umożliwi to nam, osiągniecie lepszych wyników. Na potrzeby eksperymentu $pm$ będzie przyjmowało kolejno wartości równe $0,1$, $0,2$ oraz $0,3$.

In [78]:
solver = GeneticAlgorithmSolver(pm=0.1)

In [79]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator ,problem=problem, n_samples=30)

In [80]:
analyzer.analyze()

Sample 1: Best score = 1926
Sample 2: Best score = 1924
Sample 3: Best score = 1925
Sample 4: Best score = 1926
Sample 5: Best score = 1925
Sample 6: Best score = 1926
Sample 7: Best score = 1925
Sample 8: Best score = 1925
Sample 9: Best score = 1927
Sample 10: Best score = 1924
Sample 11: Best score = 1927
Sample 12: Best score = 1925
Sample 13: Best score = 1925
Sample 14: Best score = 1928
Sample 15: Best score = 1928
Sample 16: Best score = 1927
Sample 17: Best score = 1926
Sample 18: Best score = 1925
Sample 19: Best score = 1926
Sample 20: Best score = 1925
Sample 21: Best score = 1927
Sample 22: Best score = 1925
Sample 23: Best score = 1926
Sample 24: Best score = 1926
Sample 25: Best score = 1926
Sample 26: Best score = 1925
Sample 27: Best score = 1926
Sample 28: Best score = 1927
Sample 29: Best score = 1926
Sample 30: Best score = 1925
 
Best sample = 14
Best score = 1928, Minimum score = 1924
Average score = 1925.8, Median score = 1926.0, Standard deviation = 1.0132456102

In [84]:
solver = GeneticAlgorithmSolver(pm=0.2)

In [85]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator ,problem=problem, n_samples=30)

In [86]:
analyzer.analyze()

Sample 1: Best score = 1924
Sample 2: Best score = 1926
Sample 3: Best score = 1927
Sample 4: Best score = 1924
Sample 5: Best score = 1926
Sample 6: Best score = 1924
Sample 7: Best score = 1926
Sample 8: Best score = 1925
Sample 9: Best score = 1926
Sample 10: Best score = 1923
Sample 11: Best score = 1926
Sample 12: Best score = 1925
Sample 13: Best score = 1924
Sample 14: Best score = 1926
Sample 15: Best score = 1924
Sample 16: Best score = 1925
Sample 17: Best score = 1926
Sample 18: Best score = 1925
Sample 19: Best score = 1926
Sample 20: Best score = 1928
Sample 21: Best score = 1925
Sample 22: Best score = 1928
Sample 23: Best score = 1925
Sample 24: Best score = 1928
Sample 25: Best score = 1924
Sample 26: Best score = 1924
Sample 27: Best score = 1925
Sample 28: Best score = 1924
Sample 29: Best score = 1924
Sample 30: Best score = 1925
 
Best sample = 20
Best score = 1928, Minimum score = 1923
Average score = 1925.2666666666667, Median score = 1925.0, Standard deviation = 

In [90]:
solver = GeneticAlgorithmSolver(pm=0.3)

In [91]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator ,problem=problem, n_samples=30)

In [92]:
analyzer.analyze()

Sample 1: Best score = 1925
Sample 2: Best score = 1925
Sample 3: Best score = 1924
Sample 4: Best score = 1925
Sample 5: Best score = 1923
Sample 6: Best score = 1924
Sample 7: Best score = 1925
Sample 8: Best score = 1931
Sample 9: Best score = 1924
Sample 10: Best score = 1924
Sample 11: Best score = 1925
Sample 12: Best score = 1925
Sample 13: Best score = 1925
Sample 14: Best score = 1923
Sample 15: Best score = 1925
Sample 16: Best score = 1925
Sample 17: Best score = 1928
Sample 18: Best score = 1924
Sample 19: Best score = 1924
Sample 20: Best score = 1924
Sample 21: Best score = 1924
Sample 22: Best score = 1925
Sample 23: Best score = 1925
Sample 24: Best score = 1924
Sample 25: Best score = 1925
Sample 26: Best score = 1925
Sample 27: Best score = 1925
Sample 28: Best score = 1924
Sample 29: Best score = 1924
Sample 30: Best score = 1924
 
Best sample = 8
Best score = 1931, Minimum score = 1923
Average score = 1924.7666666666667, Median score = 1925.0, Standard deviation = 1

Otrzymane średnie wyniki są zauważalnie gorsze od średniego wyniku, który otrzymaliśmy dla $pm = 0,05$. Dodatkowo zauważamy, że wraz ze wzrostem $pm$ rezultaty stają się coraz gorsze, co wskazuje, że osobniki są zaburzane w zbyt dużym stopniu. Aby potwierdzić nasze obserwacje, przeprowadźmy dodatkowo analizę dla $pm$ równego $0,5$ oraz $0,9$.

In [9]:
solver = GeneticAlgorithmSolver(pm=0.5)

In [10]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator ,problem=problem, n_samples=30)

In [11]:
analyzer.analyze()

Sample 1: Best score = 1925
Sample 2: Best score = 1924
Sample 3: Best score = 1927
Sample 4: Best score = 1925
Sample 5: Best score = 1924
Sample 6: Best score = 1924
Sample 7: Best score = 1926
Sample 8: Best score = 1926
Sample 9: Best score = 1924
Sample 10: Best score = 1924
Sample 11: Best score = 1923
Sample 12: Best score = 1923
Sample 13: Best score = 1925
Sample 14: Best score = 1925
Sample 15: Best score = 1925
Sample 16: Best score = 1923
Sample 17: Best score = 1924
Sample 18: Best score = 1926
Sample 19: Best score = 1923
Sample 20: Best score = 1925
Sample 21: Best score = 1924
Sample 22: Best score = 1924
Sample 23: Best score = 1927
Sample 24: Best score = 1926
Sample 25: Best score = 1928
Sample 26: Best score = 1927
Sample 27: Best score = 1924
Sample 28: Best score = 1926
Sample 29: Best score = 1926
Sample 30: Best score = 1923
 
Best sample = 25
Best score = 1928, Minimum score = 1923
Average score = 1924.8666666666666, Median score = 1925.0, Standard deviation = 

In [96]:
solver = GeneticAlgorithmSolver(pm=0.9)

In [97]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator ,problem=problem, n_samples=30)

In [98]:
analyzer.analyze()

Sample 1: Best score = 1928
Sample 2: Best score = 1924
Sample 3: Best score = 1924
Sample 4: Best score = 1926
Sample 5: Best score = 1925
Sample 6: Best score = 1924
Sample 7: Best score = 1923
Sample 8: Best score = 1925
Sample 9: Best score = 1924
Sample 10: Best score = 1928
Sample 11: Best score = 1922
Sample 12: Best score = 1924
Sample 13: Best score = 1925
Sample 14: Best score = 1927
Sample 15: Best score = 1924
Sample 16: Best score = 1926
Sample 17: Best score = 1926
Sample 18: Best score = 1923
Sample 19: Best score = 1923
Sample 20: Best score = 1923
Sample 21: Best score = 1924
Sample 22: Best score = 1927
Sample 23: Best score = 1926
Sample 24: Best score = 1925
Sample 25: Best score = 1926
Sample 26: Best score = 1925
Sample 27: Best score = 1925
Sample 28: Best score = 1924
Sample 29: Best score = 1924
Sample 30: Best score = 1925
 
Best sample = 1
Best score = 1928, Minimum score = 1922
Average score = 1924.8333333333333, Median score = 1925.0, Standard deviation = 1

Podobnie jak w poprzednich trzech przypadkach, rezultaty są gorsze od tych, które otrzymaliśmy dla $pm = 0,05$.

Wiedząc, że zwiększanie wartości $pm$ względem domyślnej nie przynosi poprawy, spróbujmy zmniejszyć wartość $pm$, zmniejszając tym samym różnorodność osobników. Analizę przeprowadzimy dla $pm$ równego $0,01$, $0,005$ oraz $0,001$.

In [102]:
solver = GeneticAlgorithmSolver(pm=0.01)

In [103]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [104]:
analyzer.analyze()

Sample 1: Best score = 1925
Sample 2: Best score = 1928
Sample 3: Best score = 1926
Sample 4: Best score = 1927
Sample 5: Best score = 1925
Sample 6: Best score = 1927
Sample 7: Best score = 1928
Sample 8: Best score = 1925
Sample 9: Best score = 1925
Sample 10: Best score = 1925
Sample 11: Best score = 1925
Sample 12: Best score = 1925
Sample 13: Best score = 1925
Sample 14: Best score = 1926
Sample 15: Best score = 1925
Sample 16: Best score = 1926
Sample 17: Best score = 1924
Sample 18: Best score = 1926
Sample 19: Best score = 1927
Sample 20: Best score = 1925
Sample 21: Best score = 1927
Sample 22: Best score = 1928
Sample 23: Best score = 1926
Sample 24: Best score = 1927
Sample 25: Best score = 1927
Sample 26: Best score = 1927
Sample 27: Best score = 1927
Sample 28: Best score = 1924
Sample 29: Best score = 1926
Sample 30: Best score = 1927
 
Best sample = 2
Best score = 1928, Minimum score = 1924
Average score = 1926.0333333333333, Median score = 1926.0, Standard deviation = 1

In [108]:
solver = GeneticAlgorithmSolver(pm=0.005)

In [109]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [110]:
analyzer.analyze()

Sample 1: Best score = 1926
Sample 2: Best score = 1924
Sample 3: Best score = 1929
Sample 4: Best score = 1924
Sample 5: Best score = 1925
Sample 6: Best score = 1926
Sample 7: Best score = 1924
Sample 8: Best score = 1925
Sample 9: Best score = 1927
Sample 10: Best score = 1926
Sample 11: Best score = 1926
Sample 12: Best score = 1926
Sample 13: Best score = 1926
Sample 14: Best score = 1925
Sample 15: Best score = 1927
Sample 16: Best score = 1927
Sample 17: Best score = 1924
Sample 18: Best score = 1925
Sample 19: Best score = 1926
Sample 20: Best score = 1925
Sample 21: Best score = 1928
Sample 22: Best score = 1925
Sample 23: Best score = 1925
Sample 24: Best score = 1926
Sample 25: Best score = 1926
Sample 26: Best score = 1926
Sample 27: Best score = 1926
Sample 28: Best score = 1923
Sample 29: Best score = 1926
Sample 30: Best score = 1925
 
Best sample = 3
Best score = 1929, Minimum score = 1923
Average score = 1925.6333333333334, Median score = 1926.0, Standard deviation = 1

In [15]:
solver = GeneticAlgorithmSolver(pm=0.001)

In [16]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [17]:
analyzer.analyze()

Sample 1: Best score = 1925
Sample 2: Best score = 1925
Sample 3: Best score = 1923
Sample 4: Best score = 1923
Sample 5: Best score = 1926
Sample 6: Best score = 1925
Sample 7: Best score = 1924
Sample 8: Best score = 1925
Sample 9: Best score = 1926
Sample 10: Best score = 1923
Sample 11: Best score = 1925
Sample 12: Best score = 1927
Sample 13: Best score = 1923
Sample 14: Best score = 1925
Sample 15: Best score = 1925
Sample 16: Best score = 1924
Sample 17: Best score = 1928
Sample 18: Best score = 1925
Sample 19: Best score = 1924
Sample 20: Best score = 1926
Sample 21: Best score = 1925
Sample 22: Best score = 1922
Sample 23: Best score = 1927
Sample 24: Best score = 1926
Sample 25: Best score = 1922
Sample 26: Best score = 1920
Sample 27: Best score = 1927
Sample 28: Best score = 1924
Sample 29: Best score = 1924
Sample 30: Best score = 1923
 
Best sample = 17
Best score = 1928, Minimum score = 1920
Average score = 1924.5666666666666, Median score = 1925.0, Standard deviation = 

I tym razem otrzymane rezultaty są gorsze od tych, które otrzymaliśmy dla domyślnej wartości $pm$. O ile jednak końcowy wynik dla $pm = 0,01$ jest bardzo dobry, o tyle dla $pm = 0,001$ otrzymany rezultat odbiega od tego dla $pm = 0.05$, jak i tych wygenerowanych dla $pm = 0.01$ i $pm = 0.005$.

Na koniec sprawdźmy, co się stanie, gdy $pm$ będzie równe 0. Czy i w tym przypadku rezultat znowu się pogorszy?

In [28]:
solver = GeneticAlgorithmSolver(pm=0)

In [29]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [30]:
analyzer.analyze()

Sample 1: Best score = 1920
Sample 2: Best score = 1919
Sample 3: Best score = 1924
Sample 4: Best score = 1922
Sample 5: Best score = 1923
Sample 6: Best score = 1918
Sample 7: Best score = 1920
Sample 8: Best score = 1922
Sample 9: Best score = 1922
Sample 10: Best score = 1918
Sample 11: Best score = 1924
Sample 12: Best score = 1918
Sample 13: Best score = 1920
Sample 14: Best score = 1919
Sample 15: Best score = 1921
Sample 16: Best score = 1923
Sample 17: Best score = 1921
Sample 18: Best score = 1922
Sample 19: Best score = 1921
Sample 20: Best score = 1919
Sample 21: Best score = 1922
Sample 22: Best score = 1922
Sample 23: Best score = 1918
Sample 24: Best score = 1922
Sample 25: Best score = 1921
Sample 26: Best score = 1922
Sample 27: Best score = 1921
Sample 28: Best score = 1921
Sample 29: Best score = 1920
Sample 30: Best score = 1920
 
Best sample = 3
Best score = 1924, Minimum score = 1918
Average score = 1920.8333333333333, Median score = 1921.0, Standard deviation = 1

Jak łatwo zauważyć średni wynik dla $pm = 0$ ($1920,8333$) jest o wiele gorszy od tego, który otrzymaliśmy dla $pm = 0,05$ ($1926,6$). Różnica jest w przybliżeniu równa $6$. Oprócz tego widać jak duży wpływ na wynik ma sama obecność mutacji. Nawet przy prawdopodobieństwie mutacji równym $0,001$ wyniki są zauważalnie lepsze niż w przypadku, gdy prawdopodobieństwo to jest równe $0$ (różnica bliska $4$).

Poniżej zamieszczam wyniki z przeprowadzonych do tej pory doświadczeń:

| t_max | $pm$  | Średni wynik | Mediana | Odchylenie standardowe | Max  | Min  |
|-------|-------|--------------|---------|------------------------|------|------|
| 1000  | 0     | 1920.8333    | 1921    | 1.6948                 | 1924 | 1918 |
| 1000  | 0.001 | 1924.5667    | 1925    | 1.7065                 | 1928 | 1920 |
| 1000  | 0.005 | 1925.6333    | 1926    | 1.2242                 | 1929 | 1923 |
| 1000  | 0.01  | 1926.0333    | 1926    | 1.1397                 | 1928 | 1924 |
| 1000  | 0.05  | 1926.6       | 1926    | 1.9253                 | 1932 | 1924 |
| 1000  | 0.1   | 1925.8       | 1926    | 1.0132                 | 1928 | 1924 |
| 1000  | 0.2   | 1925.2667    | 1925    | 1.2893                 | 1928 | 1923 |
| 1000  | 0.3   | 1924.7667    | 1925    | 1.4533                 | 1931 | 1923 |
| 1000  | 0.5   | 1924.8667    | 1925    | 1.3597                 | 1928 | 1923 |
| 1000  | 0.9   | 1924.7333    | 1924    | 1.2632                 | 1928 | 1923 |


| t_max   | Średni wynik (średnia) | Mediana (średnia) | Odchylenie standardowe (średnia) | Max (max) | Min (min) |
|---------|------------------------|-------------------|----------------------------------|-----------|-----------|
| 1000    | 1924.91                | 1924.9            | 1.4069                           | 1932      | 1918      |

#### 5.2 Badanie wpływu prawdopodobieństwa mutacji dla $t_{max} = 50$

W celu zachowania spójności rozważań doświadczenia zostaną przeprowadzone dla tych samych wartości hiperparametru $pm$, zmniejszona zostanie jedynie liczba iteracji w przebiegu do $50$.

In [12]:
solver = GeneticAlgorithmSolver(pm=0, t_max=50)

In [13]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [14]:
analyzer.analyze()

Sample 1: Best score = 1923
Sample 2: Best score = 1919
Sample 3: Best score = 1918
Sample 4: Best score = 1920
Sample 5: Best score = 1921
Sample 6: Best score = 1917
Sample 7: Best score = 1920
Sample 8: Best score = 1920
Sample 9: Best score = 1920
Sample 10: Best score = 1918
Sample 11: Best score = 1919
Sample 12: Best score = 1919
Sample 13: Best score = 1923
Sample 14: Best score = 1920
Sample 15: Best score = 1920
Sample 16: Best score = 1919
Sample 17: Best score = 1921
Sample 18: Best score = 1921
Sample 19: Best score = 1920
Sample 20: Best score = 1918
Sample 21: Best score = 1919
Sample 22: Best score = 1921
Sample 23: Best score = 1919
Sample 24: Best score = 1918
Sample 25: Best score = 1919
Sample 26: Best score = 1920
Sample 27: Best score = 1918
Sample 28: Best score = 1922
Sample 29: Best score = 1919
Sample 30: Best score = 1920
 
Best sample = 1
Best score = 1923, Minimum score = 1917
Average score = 1919.7, Median score = 1920.0, Standard deviation = 1.41774468787

In [39]:
solver = GeneticAlgorithmSolver(pm=0.001, t_max=50)

In [40]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [41]:
analyzer.analyze()

Sample 1: Best score = 1921
Sample 2: Best score = 1923
Sample 3: Best score = 1922
Sample 4: Best score = 1922
Sample 5: Best score = 1922
Sample 6: Best score = 1920
Sample 7: Best score = 1918
Sample 8: Best score = 1918
Sample 9: Best score = 1921
Sample 10: Best score = 1919
Sample 11: Best score = 1922
Sample 12: Best score = 1922
Sample 13: Best score = 1919
Sample 14: Best score = 1919
Sample 15: Best score = 1921
Sample 16: Best score = 1918
Sample 17: Best score = 1924
Sample 18: Best score = 1918
Sample 19: Best score = 1920
Sample 20: Best score = 1919
Sample 21: Best score = 1921
Sample 22: Best score = 1923
Sample 23: Best score = 1920
Sample 24: Best score = 1920
Sample 25: Best score = 1919
Sample 26: Best score = 1921
Sample 27: Best score = 1921
Sample 28: Best score = 1921
Sample 29: Best score = 1921
Sample 30: Best score = 1918
 
Best sample = 17
Best score = 1924, Minimum score = 1918
Average score = 1920.4333333333334, Median score = 1921.0, Standard deviation = 

In [36]:
solver = GeneticAlgorithmSolver(pm=0.005, t_max=50)

In [37]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [38]:
analyzer.analyze()

Sample 1: Best score = 1925
Sample 2: Best score = 1922
Sample 3: Best score = 1921
Sample 4: Best score = 1919
Sample 5: Best score = 1920
Sample 6: Best score = 1917
Sample 7: Best score = 1921
Sample 8: Best score = 1921
Sample 9: Best score = 1920
Sample 10: Best score = 1922
Sample 11: Best score = 1920
Sample 12: Best score = 1919
Sample 13: Best score = 1918
Sample 14: Best score = 1920
Sample 15: Best score = 1921
Sample 16: Best score = 1920
Sample 17: Best score = 1920
Sample 18: Best score = 1920
Sample 19: Best score = 1920
Sample 20: Best score = 1921
Sample 21: Best score = 1919
Sample 22: Best score = 1923
Sample 23: Best score = 1918
Sample 24: Best score = 1919
Sample 25: Best score = 1920
Sample 26: Best score = 1921
Sample 27: Best score = 1919
Sample 28: Best score = 1922
Sample 29: Best score = 1921
Sample 30: Best score = 1920
 
Best sample = 1
Best score = 1925, Minimum score = 1917
Average score = 1920.3, Median score = 1920.0, Standard deviation = 1.55241746962

In [33]:
solver = GeneticAlgorithmSolver(pm=0.01, t_max=50)

In [34]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [35]:
analyzer.analyze()

Sample 1: Best score = 1920
Sample 2: Best score = 1924
Sample 3: Best score = 1920
Sample 4: Best score = 1918
Sample 5: Best score = 1921
Sample 6: Best score = 1920
Sample 7: Best score = 1926
Sample 8: Best score = 1919
Sample 9: Best score = 1922
Sample 10: Best score = 1922
Sample 11: Best score = 1921
Sample 12: Best score = 1919
Sample 13: Best score = 1921
Sample 14: Best score = 1921
Sample 15: Best score = 1921
Sample 16: Best score = 1920
Sample 17: Best score = 1920
Sample 18: Best score = 1920
Sample 19: Best score = 1923
Sample 20: Best score = 1922
Sample 21: Best score = 1920
Sample 22: Best score = 1923
Sample 23: Best score = 1922
Sample 24: Best score = 1920
Sample 25: Best score = 1923
Sample 26: Best score = 1921
Sample 27: Best score = 1923
Sample 28: Best score = 1924
Sample 29: Best score = 1924
Sample 30: Best score = 1921
 
Best sample = 7
Best score = 1926, Minimum score = 1918
Average score = 1921.3666666666666, Median score = 1921.0, Standard deviation = 1

In [30]:
solver = GeneticAlgorithmSolver(t_max=50)

In [31]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [32]:
analyzer.analyze()

Sample 1: Best score = 1922
Sample 2: Best score = 1922
Sample 3: Best score = 1920
Sample 4: Best score = 1925
Sample 5: Best score = 1922
Sample 6: Best score = 1922
Sample 7: Best score = 1924
Sample 8: Best score = 1923
Sample 9: Best score = 1921
Sample 10: Best score = 1924
Sample 11: Best score = 1921
Sample 12: Best score = 1922
Sample 13: Best score = 1922
Sample 14: Best score = 1921
Sample 15: Best score = 1920
Sample 16: Best score = 1922
Sample 17: Best score = 1921
Sample 18: Best score = 1924
Sample 19: Best score = 1923
Sample 20: Best score = 1924
Sample 21: Best score = 1923
Sample 22: Best score = 1923
Sample 23: Best score = 1924
Sample 24: Best score = 1923
Sample 25: Best score = 1924
Sample 26: Best score = 1922
Sample 27: Best score = 1921
Sample 28: Best score = 1923
Sample 29: Best score = 1920
Sample 30: Best score = 1923
 
Best sample = 4
Best score = 1925, Minimum score = 1920
Average score = 1922.3666666666666, Median score = 1922.0, Standard deviation = 1

In [15]:
solver = GeneticAlgorithmSolver(pm=0.1, t_max=50)

In [16]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [17]:
analyzer.analyze()

Sample 1: Best score = 1921
Sample 2: Best score = 1926
Sample 3: Best score = 1923
Sample 4: Best score = 1923
Sample 5: Best score = 1920
Sample 6: Best score = 1920
Sample 7: Best score = 1922
Sample 8: Best score = 1923
Sample 9: Best score = 1920
Sample 10: Best score = 1926
Sample 11: Best score = 1925
Sample 12: Best score = 1921
Sample 13: Best score = 1922
Sample 14: Best score = 1922
Sample 15: Best score = 1920
Sample 16: Best score = 1920
Sample 17: Best score = 1924
Sample 18: Best score = 1922
Sample 19: Best score = 1926
Sample 20: Best score = 1922
Sample 21: Best score = 1922
Sample 22: Best score = 1920
Sample 23: Best score = 1923
Sample 24: Best score = 1921
Sample 25: Best score = 1922
Sample 26: Best score = 1920
Sample 27: Best score = 1921
Sample 28: Best score = 1923
Sample 29: Best score = 1924
Sample 30: Best score = 1920
 
Best sample = 2
Best score = 1926, Minimum score = 1920
Average score = 1922.1333333333334, Median score = 1922.0, Standard deviation = 1

In [18]:
solver = GeneticAlgorithmSolver(pm=0.2, t_max=50)

In [19]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [20]:
analyzer.analyze()

Sample 1: Best score = 1921
Sample 2: Best score = 1922
Sample 3: Best score = 1923
Sample 4: Best score = 1922
Sample 5: Best score = 1921
Sample 6: Best score = 1921
Sample 7: Best score = 1922
Sample 8: Best score = 1920
Sample 9: Best score = 1919
Sample 10: Best score = 1922
Sample 11: Best score = 1923
Sample 12: Best score = 1921
Sample 13: Best score = 1924
Sample 14: Best score = 1921
Sample 15: Best score = 1922
Sample 16: Best score = 1922
Sample 17: Best score = 1925
Sample 18: Best score = 1921
Sample 19: Best score = 1921
Sample 20: Best score = 1922
Sample 21: Best score = 1922
Sample 22: Best score = 1923
Sample 23: Best score = 1924
Sample 24: Best score = 1922
Sample 25: Best score = 1923
Sample 26: Best score = 1920
Sample 27: Best score = 1920
Sample 28: Best score = 1921
Sample 29: Best score = 1919
Sample 30: Best score = 1920
 
Best sample = 17
Best score = 1925, Minimum score = 1919
Average score = 1921.6333333333334, Median score = 1922.0, Standard deviation = 

In [21]:
solver = GeneticAlgorithmSolver(pm=0.3, t_max=50)

In [22]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [23]:
analyzer.analyze()

Sample 1: Best score = 1923
Sample 2: Best score = 1922
Sample 3: Best score = 1923
Sample 4: Best score = 1920
Sample 5: Best score = 1920
Sample 6: Best score = 1922
Sample 7: Best score = 1921
Sample 8: Best score = 1922
Sample 9: Best score = 1921
Sample 10: Best score = 1922
Sample 11: Best score = 1923
Sample 12: Best score = 1923
Sample 13: Best score = 1922
Sample 14: Best score = 1919
Sample 15: Best score = 1926
Sample 16: Best score = 1921
Sample 17: Best score = 1921
Sample 18: Best score = 1921
Sample 19: Best score = 1924
Sample 20: Best score = 1922
Sample 21: Best score = 1921
Sample 22: Best score = 1921
Sample 23: Best score = 1920
Sample 24: Best score = 1921
Sample 25: Best score = 1922
Sample 26: Best score = 1925
Sample 27: Best score = 1920
Sample 28: Best score = 1920
Sample 29: Best score = 1922
Sample 30: Best score = 1922
 
Best sample = 15
Best score = 1926, Minimum score = 1919
Average score = 1921.7333333333333, Median score = 1922.0, Standard deviation = 

In [24]:
solver = GeneticAlgorithmSolver(pm=0.5, t_max=50)

In [25]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [26]:
analyzer.analyze()

Sample 1: Best score = 1921
Sample 2: Best score = 1921
Sample 3: Best score = 1921
Sample 4: Best score = 1921
Sample 5: Best score = 1921
Sample 6: Best score = 1923
Sample 7: Best score = 1920
Sample 8: Best score = 1920
Sample 9: Best score = 1925
Sample 10: Best score = 1923
Sample 11: Best score = 1921
Sample 12: Best score = 1920
Sample 13: Best score = 1921
Sample 14: Best score = 1921
Sample 15: Best score = 1922
Sample 16: Best score = 1919
Sample 17: Best score = 1923
Sample 18: Best score = 1921
Sample 19: Best score = 1921
Sample 20: Best score = 1921
Sample 21: Best score = 1920
Sample 22: Best score = 1920
Sample 23: Best score = 1921
Sample 24: Best score = 1922
Sample 25: Best score = 1922
Sample 26: Best score = 1920
Sample 27: Best score = 1923
Sample 28: Best score = 1919
Sample 29: Best score = 1923
Sample 30: Best score = 1920
 
Best sample = 9
Best score = 1925, Minimum score = 1919
Average score = 1921.2, Median score = 1921.0, Standard deviation = 1.32664991614

In [27]:
solver = GeneticAlgorithmSolver(pm=0.9, t_max=50)

In [28]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [29]:
analyzer.analyze()

Sample 1: Best score = 1919
Sample 2: Best score = 1921
Sample 3: Best score = 1920
Sample 4: Best score = 1921
Sample 5: Best score = 1919
Sample 6: Best score = 1921
Sample 7: Best score = 1922
Sample 8: Best score = 1922
Sample 9: Best score = 1920
Sample 10: Best score = 1921
Sample 11: Best score = 1920
Sample 12: Best score = 1926
Sample 13: Best score = 1921
Sample 14: Best score = 1922
Sample 15: Best score = 1921
Sample 16: Best score = 1919
Sample 17: Best score = 1922
Sample 18: Best score = 1920
Sample 19: Best score = 1922
Sample 20: Best score = 1920
Sample 21: Best score = 1922
Sample 22: Best score = 1918
Sample 23: Best score = 1920
Sample 24: Best score = 1920
Sample 25: Best score = 1920
Sample 26: Best score = 1921
Sample 27: Best score = 1925
Sample 28: Best score = 1920
Sample 29: Best score = 1921
Sample 30: Best score = 1921
 
Best sample = 12
Best score = 1926, Minimum score = 1918
Average score = 1920.9, Median score = 1921.0, Standard deviation = 1.5989579940

Poniżej zamieszczam wyniki z przeprowadzonych doświadczeń dla $t_{max} = 50$:

| t_max  | $pm$  | Średni wynik   | Mediana   | Odchylenie standardowe   | Max  | Min  |
|--------|-------|----------------|-----------|--------------------------|------|------|
| 50     | 0     | 1919.7         | 1920      | 1.4177                   | 1923 | 1917 |
| 50     | 0.001 | 1920.4333      | 1921      | 1.6469                   | 1924 | 1918 |
| 50     | 0.005 | 1920.3         | 1920      | 1.5524                   | 1925 | 1917 |
| 50     | 0.01  | 1921.3667      | 1921      | 1.7604                   | 1926 | 1918 |
| 50     | 0.05  | 1922.3667      | 1922      | 1.3287                   | 1925 | 1920 |
| 50     | 0.1   | 1922.1333      | 1922      | 1.8749                   | 1926 | 1920 |
| 50     | 0.2   | 1921.6333      | 1922      | 1.4019                   | 1925 | 1919 |
| 50     | 0.3   | 1921.7333      | 1922      | 1.5041                   | 1926 | 1919 |
| 50     | 0.5   | 1921.2         | 1921      | 1.3266                   | 1925 | 1919 |
| 50     | 0.9   | 1920.9         | 1921      | 1.5989                   | 1926 | 1918 |


| t_max  | Średni wynik (średnia)   | Mediana (średnia)   | Odchylenie standardowe (średnia)   | Max (max) | Min (min) |
|--------|--------------------------|---------------------|------------------------------------|-----------|-----------|
| 50     | 1921.1767                | 1921.2              | 1.5413                             | 1926      | 1917      |

#### 5.3 Badanie wpływu prawdopodobieństwa mutacji dla $t_{max} = 100$

W tym podpunkcie będziemy badali wpływ hiperparametru $pm$ dla $t_{max} = 100$. Rozważania są przeprowadzane analogicznie jak w poprzednich podpunktach.

In [42]:
solver = GeneticAlgorithmSolver(pm=0, t_max=100)

In [43]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [44]:
analyzer.analyze()

Sample 1: Best score = 1919
Sample 2: Best score = 1918
Sample 3: Best score = 1918
Sample 4: Best score = 1919
Sample 5: Best score = 1918
Sample 6: Best score = 1921
Sample 7: Best score = 1920
Sample 8: Best score = 1920
Sample 9: Best score = 1921
Sample 10: Best score = 1922
Sample 11: Best score = 1920
Sample 12: Best score = 1918
Sample 13: Best score = 1920
Sample 14: Best score = 1921
Sample 15: Best score = 1922
Sample 16: Best score = 1921
Sample 17: Best score = 1919
Sample 18: Best score = 1917
Sample 19: Best score = 1920
Sample 20: Best score = 1919
Sample 21: Best score = 1922
Sample 22: Best score = 1920
Sample 23: Best score = 1923
Sample 24: Best score = 1921
Sample 25: Best score = 1921
Sample 26: Best score = 1922
Sample 27: Best score = 1921
Sample 28: Best score = 1923
Sample 29: Best score = 1919
Sample 30: Best score = 1921
 
Best sample = 23
Best score = 1923, Minimum score = 1917
Average score = 1920.2, Median score = 1920.0, Standard deviation = 1.5362291495

In [72]:
solver = GeneticAlgorithmSolver(pm=0.001, t_max=100)

In [73]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [74]:
analyzer.analyze()

Sample 1: Best score = 1922
Sample 2: Best score = 1918
Sample 3: Best score = 1918
Sample 4: Best score = 1923
Sample 5: Best score = 1918
Sample 6: Best score = 1919
Sample 7: Best score = 1921
Sample 8: Best score = 1920
Sample 9: Best score = 1919
Sample 10: Best score = 1920
Sample 11: Best score = 1919
Sample 12: Best score = 1920
Sample 13: Best score = 1921
Sample 14: Best score = 1923
Sample 15: Best score = 1921
Sample 16: Best score = 1921
Sample 17: Best score = 1921
Sample 18: Best score = 1919
Sample 19: Best score = 1919
Sample 20: Best score = 1920
Sample 21: Best score = 1921
Sample 22: Best score = 1920
Sample 23: Best score = 1922
Sample 24: Best score = 1927
Sample 25: Best score = 1922
Sample 26: Best score = 1923
Sample 27: Best score = 1922
Sample 28: Best score = 1919
Sample 29: Best score = 1923
Sample 30: Best score = 1923
 
Best sample = 24
Best score = 1927, Minimum score = 1918
Average score = 1920.8, Median score = 1921.0, Standard deviation = 1.9561867668

In [69]:
solver = GeneticAlgorithmSolver(pm=0.005, t_max=100)

In [70]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [71]:
analyzer.analyze()

Sample 1: Best score = 1924
Sample 2: Best score = 1923
Sample 3: Best score = 1923
Sample 4: Best score = 1922
Sample 5: Best score = 1920
Sample 6: Best score = 1922
Sample 7: Best score = 1921
Sample 8: Best score = 1925
Sample 9: Best score = 1926
Sample 10: Best score = 1923
Sample 11: Best score = 1922
Sample 12: Best score = 1922
Sample 13: Best score = 1922
Sample 14: Best score = 1924
Sample 15: Best score = 1921
Sample 16: Best score = 1922
Sample 17: Best score = 1921
Sample 18: Best score = 1921
Sample 19: Best score = 1920
Sample 20: Best score = 1922
Sample 21: Best score = 1924
Sample 22: Best score = 1920
Sample 23: Best score = 1922
Sample 24: Best score = 1921
Sample 25: Best score = 1925
Sample 26: Best score = 1922
Sample 27: Best score = 1925
Sample 28: Best score = 1921
Sample 29: Best score = 1923
Sample 30: Best score = 1922
 
Best sample = 9
Best score = 1926, Minimum score = 1920
Average score = 1922.3666666666666, Median score = 1922.0, Standard deviation = 1

In [66]:
solver = GeneticAlgorithmSolver(pm=0.01, t_max=100)

In [67]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [68]:
analyzer.analyze()

Sample 1: Best score = 1922
Sample 2: Best score = 1922
Sample 3: Best score = 1922
Sample 4: Best score = 1920
Sample 5: Best score = 1925
Sample 6: Best score = 1922
Sample 7: Best score = 1922
Sample 8: Best score = 1922
Sample 9: Best score = 1928
Sample 10: Best score = 1922
Sample 11: Best score = 1925
Sample 12: Best score = 1924
Sample 13: Best score = 1922
Sample 14: Best score = 1923
Sample 15: Best score = 1921
Sample 16: Best score = 1921
Sample 17: Best score = 1921
Sample 18: Best score = 1926
Sample 19: Best score = 1923
Sample 20: Best score = 1925
Sample 21: Best score = 1925
Sample 22: Best score = 1921
Sample 23: Best score = 1921
Sample 24: Best score = 1924
Sample 25: Best score = 1926
Sample 26: Best score = 1921
Sample 27: Best score = 1925
Sample 28: Best score = 1925
Sample 29: Best score = 1924
Sample 30: Best score = 1923
 
Best sample = 9
Best score = 1928, Minimum score = 1920
Average score = 1923.1, Median score = 1922.5, Standard deviation = 1.92093727122

In [63]:
solver = GeneticAlgorithmSolver(t_max=100)

In [64]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [65]:
analyzer.analyze()

Sample 1: Best score = 1922
Sample 2: Best score = 1925
Sample 3: Best score = 1923
Sample 4: Best score = 1923
Sample 5: Best score = 1923
Sample 6: Best score = 1923
Sample 7: Best score = 1924
Sample 8: Best score = 1923
Sample 9: Best score = 1920
Sample 10: Best score = 1924
Sample 11: Best score = 1926
Sample 12: Best score = 1922
Sample 13: Best score = 1925
Sample 14: Best score = 1924
Sample 15: Best score = 1923
Sample 16: Best score = 1923
Sample 17: Best score = 1924
Sample 18: Best score = 1924
Sample 19: Best score = 1923
Sample 20: Best score = 1926
Sample 21: Best score = 1922
Sample 22: Best score = 1923
Sample 23: Best score = 1924
Sample 24: Best score = 1925
Sample 25: Best score = 1924
Sample 26: Best score = 1923
Sample 27: Best score = 1923
Sample 28: Best score = 1924
Sample 29: Best score = 1922
Sample 30: Best score = 1924
 
Best sample = 11
Best score = 1926, Minimum score = 1920
Average score = 1923.4666666666667, Median score = 1923.0, Standard deviation = 

In [45]:
solver = GeneticAlgorithmSolver(pm=0.1, t_max=100)

In [46]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [47]:
analyzer.analyze()

Sample 1: Best score = 1923
Sample 2: Best score = 1921
Sample 3: Best score = 1923
Sample 4: Best score = 1925
Sample 5: Best score = 1925
Sample 6: Best score = 1923
Sample 7: Best score = 1922
Sample 8: Best score = 1923
Sample 9: Best score = 1922
Sample 10: Best score = 1924
Sample 11: Best score = 1924
Sample 12: Best score = 1923
Sample 13: Best score = 1923
Sample 14: Best score = 1923
Sample 15: Best score = 1923
Sample 16: Best score = 1925
Sample 17: Best score = 1925
Sample 18: Best score = 1923
Sample 19: Best score = 1923
Sample 20: Best score = 1926
Sample 21: Best score = 1922
Sample 22: Best score = 1921
Sample 23: Best score = 1923
Sample 24: Best score = 1923
Sample 25: Best score = 1922
Sample 26: Best score = 1924
Sample 27: Best score = 1920
Sample 28: Best score = 1926
Sample 29: Best score = 1925
Sample 30: Best score = 1923
 
Best sample = 20
Best score = 1926, Minimum score = 1920
Average score = 1923.2666666666667, Median score = 1923.0, Standard deviation = 

In [48]:
solver = GeneticAlgorithmSolver(pm=0.2, t_max=100)

In [49]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [50]:
analyzer.analyze()

Sample 1: Best score = 1922
Sample 2: Best score = 1921
Sample 3: Best score = 1921
Sample 4: Best score = 1924
Sample 5: Best score = 1923
Sample 6: Best score = 1923
Sample 7: Best score = 1921
Sample 8: Best score = 1925
Sample 9: Best score = 1921
Sample 10: Best score = 1923
Sample 11: Best score = 1921
Sample 12: Best score = 1924
Sample 13: Best score = 1924
Sample 14: Best score = 1923
Sample 15: Best score = 1923
Sample 16: Best score = 1923
Sample 17: Best score = 1922
Sample 18: Best score = 1921
Sample 19: Best score = 1922
Sample 20: Best score = 1924
Sample 21: Best score = 1923
Sample 22: Best score = 1926
Sample 23: Best score = 1925
Sample 24: Best score = 1922
Sample 25: Best score = 1921
Sample 26: Best score = 1924
Sample 27: Best score = 1921
Sample 28: Best score = 1921
Sample 29: Best score = 1922
Sample 30: Best score = 1921
 
Best sample = 22
Best score = 1926, Minimum score = 1921
Average score = 1922.5666666666666, Median score = 1922.5, Standard deviation = 

In [51]:
solver = GeneticAlgorithmSolver(pm=0.3, t_max=100)

In [52]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [53]:
analyzer.analyze()

Sample 1: Best score = 1921
Sample 2: Best score = 1921
Sample 3: Best score = 1923
Sample 4: Best score = 1919
Sample 5: Best score = 1922
Sample 6: Best score = 1921
Sample 7: Best score = 1921
Sample 8: Best score = 1924
Sample 9: Best score = 1922
Sample 10: Best score = 1921
Sample 11: Best score = 1923
Sample 12: Best score = 1923
Sample 13: Best score = 1922
Sample 14: Best score = 1925
Sample 15: Best score = 1923
Sample 16: Best score = 1923
Sample 17: Best score = 1922
Sample 18: Best score = 1923
Sample 19: Best score = 1923
Sample 20: Best score = 1922
Sample 21: Best score = 1924
Sample 22: Best score = 1922
Sample 23: Best score = 1924
Sample 24: Best score = 1923
Sample 25: Best score = 1921
Sample 26: Best score = 1921
Sample 27: Best score = 1920
Sample 28: Best score = 1922
Sample 29: Best score = 1920
Sample 30: Best score = 1922
 
Best sample = 14
Best score = 1925, Minimum score = 1919
Average score = 1922.1, Median score = 1922.0, Standard deviation = 1.3253930234

In [54]:
solver = GeneticAlgorithmSolver(pm=0.5, t_max=100)

In [55]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [56]:
analyzer.analyze()

Sample 1: Best score = 1921
Sample 2: Best score = 1921
Sample 3: Best score = 1922
Sample 4: Best score = 1921
Sample 5: Best score = 1921
Sample 6: Best score = 1927
Sample 7: Best score = 1923
Sample 8: Best score = 1922
Sample 9: Best score = 1922
Sample 10: Best score = 1920
Sample 11: Best score = 1923
Sample 12: Best score = 1922
Sample 13: Best score = 1922
Sample 14: Best score = 1924
Sample 15: Best score = 1925
Sample 16: Best score = 1920
Sample 17: Best score = 1924
Sample 18: Best score = 1922
Sample 19: Best score = 1921
Sample 20: Best score = 1922
Sample 21: Best score = 1921
Sample 22: Best score = 1922
Sample 23: Best score = 1924
Sample 24: Best score = 1922
Sample 25: Best score = 1924
Sample 26: Best score = 1920
Sample 27: Best score = 1923
Sample 28: Best score = 1924
Sample 29: Best score = 1923
Sample 30: Best score = 1922
 
Best sample = 6
Best score = 1927, Minimum score = 1920
Average score = 1922.3333333333333, Median score = 1922.0, Standard deviation = 1

In [57]:
solver = GeneticAlgorithmSolver(pm=0.9, t_max=100)

In [58]:
analyzer = GeneticAlgorithmAnalyzer(solver=solver, population_creator=population_creator, problem=problem, n_samples=30)

In [59]:
analyzer.analyze()

Sample 1: Best score = 1921
Sample 2: Best score = 1924
Sample 3: Best score = 1920
Sample 4: Best score = 1922
Sample 5: Best score = 1921
Sample 6: Best score = 1921
Sample 7: Best score = 1920
Sample 8: Best score = 1924
Sample 9: Best score = 1922
Sample 10: Best score = 1922
Sample 11: Best score = 1923
Sample 12: Best score = 1922
Sample 13: Best score = 1925
Sample 14: Best score = 1922
Sample 15: Best score = 1925
Sample 16: Best score = 1922
Sample 17: Best score = 1924
Sample 18: Best score = 1922
Sample 19: Best score = 1920
Sample 20: Best score = 1921
Sample 21: Best score = 1924
Sample 22: Best score = 1922
Sample 23: Best score = 1924
Sample 24: Best score = 1922
Sample 25: Best score = 1922
Sample 26: Best score = 1920
Sample 27: Best score = 1922
Sample 28: Best score = 1921
Sample 29: Best score = 1923
Sample 30: Best score = 1920
 
Best sample = 13
Best score = 1925, Minimum score = 1920
Average score = 1922.1, Median score = 1922.0, Standard deviation = 1.4685593847

Poniżej zamieszczam wyniki z przeprowadzonych doświadczeń dla $t_{max} = 100$:

| t_max  | $pm$  | Średni wynik  | Mediana   | Odchylenie standardowe | Max  | Min  |
|--------|-------|---------------|-----------|------------------------|------|------|
| 100    | 0     | 1920.2        | 1920      | 1.5362                 | 1923 | 1917 |
| 100    | 0.001 | 1920.8        | 1921      | 1.9562                 | 1927 | 1918 |
| 100    | 0.005 | 1922.3667     | 1922      | 1.5596                 | 1926 | 1920 |
| 100    | 0.01  | 1923.1        | 1922.5    | 1.9209                 | 1928 | 1920 |
| 100    | 0.05  | 1923.4667     | 1923      | 1.2311                 | 1926 | 1920 |
| 100    | 0.1   | 1923.2667     | 1923      | 1.4126                 | 1926 | 1920 |
| 100    | 0.2   | 1922.5667     | 1922.5    | 1.4302                 | 1926 | 1921 |
| 100    | 0.3   | 1922.1        | 1922      | 1.3254                 | 1925 | 1919 |
| 100    | 0.5   | 1922.3333     | 1922      | 1.5563                 | 1927 | 1920 |
| 100    | 0.9   | 1922.1        | 1922      | 1.4686                 | 1925 | 1920 |


| t_max  | Średni wynik (średnia) | Mediana (średnia) | Odchylenie standardowe (średnia) | Max (max) | Min (min) |
|--------|------------------------|-------------------|----------------------------------|-----------|-----------|
| 100    | 1922.23                | 1922              | 1.5397                           | 1928      | 1917      |

## 6. Podsumowanie

Przed przystąpieniem do formułowania wniosków jeszcze raz zaprezentuję zebrane dotąd rezultaty, tym razem w zbiorczej formie.

| t_max  | $pm$    | Średni wynik   | Mediana   | Odchylenie standardowe   | Max  | Min  |
|--------|---------|----------------|-----------|--------------------------|------|------|
| 50     | 0       | 1919.7         | 1920      | 1.4177                   | 1923 | 1917 |
| 50     | 0.001   | 1920.4333      | 1921      | 1.6469                   | 1924 | 1918 |
| 50     | 0.005   | 1920.3         | 1920      | 1.5524                   | 1925 | 1917 |
| 50     | 0.01    | 1921.3667      | 1921      | 1.7604                   | 1926 | 1918 |
| 50     | 0.05    | 1922.3667      | 1922      | 1.3287                   | 1925 | 1920 |
| 50     | 0.1     | 1922.1333      | 1922      | 1.8749                   | 1926 | 1920 |
| 50     | 0.2     | 1921.6333      | 1922      | 1.4019                   | 1925 | 1919 |
| 50     | 0.3     | 1921.7333      | 1922      | 1.5041                   | 1926 | 1919 |
| 50     | 0.5     | 1921.2         | 1921      | 1.3266                   | 1925 | 1919 |
| 50     | 0.9     | 1920.9         | 1921      | 1.5989                   | 1926 | 1918 |
| 100    | 0       | 1920.2         | 1920      | 1.5362                   | 1923 | 1917 |
| 100    | 0.001   | 1920.8         | 1921      | 1.9562                   | 1927 | 1918 |
| 100    | 0.005   | 1922.3667      | 1922      | 1.5596                   | 1926 | 1920 |
| 100    | 0.01    | 1923.1         | 1922.5    | 1.9209                   | 1928 | 1920 |
| 100    | 0.05    | 1923.4667      | 1923      | 1.2311                   | 1926 | 1920 |
| 100    | 0.1     | 1923.2667      | 1923      | 1.4126                   | 1926 | 1920 |
| 100    | 0.2     | 1922.5667      | 1922.5    | 1.4302                   | 1926 | 1921 |
| 100    | 0.3     | 1922.1         | 1922      | 1.3254                   | 1925 | 1919 |
| 100    | 0.5     | 1922.3333      | 1922      | 1.5563                   | 1927 | 1920 |
| 100    | 0.9     | 1922.1         | 1922      | 1.4686                   | 1925 | 1920 |
| 1000   | 0       | 1920.8333      | 1921      | 1.6948                   | 1924 | 1918 |
| 1000   | 0.001   | 1924.5667      | 1925      | 1.7065                   | 1928 | 1920 |
| 1000   | 0.005   | 1925.6333      | 1926      | 1.2242                   | 1929 | 1923 |
| 1000   | 0.01    | 1926.0333      | 1926      | 1.1397                   | 1928 | 1924 |
| 1000   | 0.05    | 1926.6         | 1926      | 1.9253                   | 1932 | 1924 |
| 1000   | 0.1     | 1925.8         | 1926      | 1.0132                   | 1928 | 1924 |
| 1000   | 0.2     | 1925.2667      | 1925      | 1.2893                   | 1928 | 1923 |
| 1000   | 0.3     | 1924.7667      | 1925      | 1.4533                   | 1931 | 1923 |
| 1000   | 0.5     | 1924.8667      | 1925      | 1.3597                   | 1928 | 1923 |
| 1000   | 0.9     | 1924.7333      | 1924      | 1.2632                   | 1928 | 1923 |


| t_max  | Średni wynik (średnia)  | Mediana (średnia) | Odchylenie standardowe (średnia) | Max (max) | Min (min) |
|--------|-------------------------|-------------------|----------------------------------|-----------|-----------|
| 50     | 1921.1767               | 1921.2            | 1.5413                           | 1926      | 1917      |
| 100    | 1922.23                 | 1922              | 1.5397                           | 1928      | 1917      |
| 1000   | 1924.91                 | 1924.9            | 1.4069                           | 1932      | 1918      |


Analizując powyższe dane, możemy stwierdzić, że zarówno dla $t_{max}$ równego $50$, $100$ jak i $1000$, wartość prawdopodobieństwa mutacji dająca najlepsze rezultaty nie może być zbyt mała, ale także zbyt duża. We wszystkich rozważanych przypadkach $t_{max}$ algorytm sprawdzał się najlepiej dla $pm = 0,05$ - naszej wartości domyślnej. Jest to wartość na tyle mała, że nie zaburza znacząco struktury osobnika, a także na tyle duża, że w znaczącym stopniu zwiększa różnorodność osobników, umożliwiając tym samym poprawienie efektywności krzyżowania w procesie wyznaczania najlepszego osobnika. W każdej z trzech przedstawionych sytuacji za każdym razem najgorsze średnie wyniki, jak i wartości minimalne otrzymywaliśmy dla $pm = 0$, co potwierdza nasze rozważania. Nawet minimalna wartość prawdopodobieństwa mutacji w zauważalny sposób poprawia wyniki, co widać szczególnie dla $t_{max} = 1000$.


Warto zauważyć także, że wraz ze wzrostem liczby iteracji w przebiegu $t_{max}$ zwiększa się także średni wynik, jak i mediana. Jest to fakt, którego mogliśmy się spodziewać od samego początku, ponieważ wraz ze wzrostem liczby iteracji, zwiększa się ilość nowych (powstałych na skutek mutacji i krzyżowania) osobników biorących udział w ewolucji, co pozytywnie wpływa na jakość populacji, która charakteryzuje się osobnikami o dużej wartości. Widzimy także, że z im większą liczbą iteracji mamy do czynienia tym średnia odchylenia standardowego staje się mniejsza. O ile dla $t_{max} = 50$ i $t_{max} = 100$ różnica jest pomijalna, o tyle dla $t_{max} = 1000$ różnica względem pozostałych przypadków jest już znacząco większa. Niestety ze względu na bardzo małe zróżnicowanie wyników w ramach jednego $t_{max}$ nie jesteśmy w stanie zaobserwować żadnej korelacji pomiędzy wartością odchylenia standardowego, a hiperparametrem $pm$, nawet po zwiększeniu liczby przebiegów (do $30$) wewnątrz jednej analizy, redukując tym samym losowość na rzecz bardziej zregularyzowanych zależności od konkretnego zestawu parametrów.

Wraz ze wzrostem liczby $t_{max}$ jesteśmy w stanie osiągnąć lepszego osobnika o maksymalnej wartości w danej analizie (dla konkretnego $t_{max}$). Dla $t_{max} = 50$ najlepszy osobnik ma wartość $1926$ dla $t_{max} = 100$ jest to już 1928, natomiast dla $t_{max} = 1000$ najlepszy wychwycony osobnik ma wartość wynoszącą aż $1932$. Analogicznie jest w przypadku najgorszych osobników. Analizy względem wartości $pm$ przeprowadzone dla $t_{max} = 50$ cechują się znacznie niższymi wartościami najgorszych osobników niż te przeprowadzone dla $t_{max} = 1000$. Jednakże to, że dana analiza charakteryzuje się najlepszym uzyskanym osobnikiem w ramach jednej wartości $t_{max}$ wcale nie świadczy, że ma ona najwyższy średni wynik. Zarówno dla $t_{max} = 50$, jak i $t_{max} = 100$ analizą, która szczyci się osobnikiem o maksymalnej wartości, nie jest ta, która została przeprowadzona dla $pm = 0,05$ lecz np. ta o $pm = 0,09$ ( $t_{max} = 50$ ) lub $pm = 0,01$ ( $t_{max} = 100$ ).

Patrząc obiektywnie na otrzymane rezultaty, możemy powiedzieć, że wszystkie wyniki są relatywnie do siebie podobne. Jest to spowodowane w głównej mierze przez samą specyfikę zadania. Gdybyśmy mieli do czynienia z zadaniem bardziej rozbudowanym, wyniki zapewne różniłyby się bardziej.